# Airline Network Revenue Management

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

In the airline network revenue management problem we are trying to decide how many tickets for each origin-destination (O-D) pair to sell at each price level. The goal is to maximize revenue, and we cannot sell more tickets than there is demand for, or space on the planes for.

## Three Flight Problem

We'll start with a toy problem that has three origin-destination pairs, with two price classes for each pair. The three origin-destination pairs are BOS-MDW, MDW-SFO, or BOS-SFO via MDW. BOS stands for Boston, MDW is Chicago-Midway, and SFO is San Francisco. Each O-D pair has a "regular" and "discount" fare class. The data we will use is summarized as follows:

```
PLANE CAPACITY: 166

BOS-MDW
        Regular  Discount
Price:  428      190
Demand: 80       120

BOS-SFO
        Regular  Discount
Price:  642      224
Demand: 75       100

MDW-SFO
        Regular  Discount
Price:  512      190
Demand: 60       110
```

What does our formulation look like?

## Variables

...

## Objective

...

## Constraints

...

# JuMP Formulation

In [ ]:
using JuMP, Gurobi
nrm = Model(solver=GurobiSolver())
@variables nrm begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end
@objective(nrm, Max, 428BOStoMDW_R + 190BOStoMDW_D +
                     642BOStoSFO_R + 224BOStoSFO_D +
                     512MDWtoSFO_R + 190MDWtoSFO_D)
@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
solve(nrm)

@show getvalue(BOStoMDW_R)
@show getvalue(BOStoMDW_D)
@show getobjectivevalue(nrm)

_What's wrong with this model?_






## Building an extensible model

First, we would like to construct a _collection of variables_ all at once.  This is a very common idiom; for example, you might have a variable named ``x`` that is indexed from 1 to 10:

In [ ]:
m = Model()
@variable(m, x[1:10] >= 0)

The index sets are specified inside the ``[...]`` block. You can create multidimensional containers by specifying multiple index sets, separated by commas:

In [ ]:
@variable(m, y[1:10,["red","blue"]] <= 1)

For more complicated expressions, you can name the indices for the index sets and use them in the rest of the variable definition:

In [ ]:
ub = rand(10)
@variable(m, i <= z[i=1:10,j=(i+1):10] <= ub[j])

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [ ]:
@variable(m, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Now that we can programatically create arrays of variables, we would like to be able to use them to full-effect in the constraints of our problem. That is, we want a way to express multi-dimensional summations, with conditionals. To do this, we use the ``sum(...)`` construction. The first argument is the ''inner loop'' of the summation, the index sets are specified after a ``for``, and any conditionals are stated following an ``if`` (similar to variable definition, but with a slightly different syntax).

In [ ]:
@constraint(m, sum(x[i] for i in 1:10) <= 1)

In [ ]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(m, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

In [ ]:
@constraint(m, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

## Revisiting the network revenue management problem

Now let's return to the network revenue management example and attempt to rewrite it in a generic way that scales to any number of airports. 

First, let's create some random data for our problem.

In [ ]:
# Set the random seed to ensure we always
# get the same stream of 'random' numbers
srand(1988)  

# Lets create a vector of symbols, one for each airport
airports = [:BOS, :MDW, :SFO, :YYZ]
num_airport = length(airports)

# We'll also create a vector of fare classes
classes = [:REG, :DIS]

# All the demand and price data for each triple of
# (origin, destination, class) will be stored in
# 'dictionaries', also known as 'maps'.
demand = Dict()
prices = Dict()

# Generate a demand and price for each pair of airports
# To keep the code simple we will generate info for
# nonsense flights like BOS-BOS and SFO-SFO, but they
# won't appear in our final model.
for origin in airports, dest in airports
    # Generate demand:
    #  - Regular demand is Uniform(50,90)
    #  - Discount demand is Uniform(100,130)
    demand[(origin,dest,:REG)] = rand(50:90)    
    demand[(origin,dest,:DIS)] = rand(100:130)
    # Generate prices:
    #  - Regular price is Uniform(400,700)
    #  - Discount price is Uniform(150,300)
    prices[(origin,dest,:REG)] = rand(400:700)
    prices[(origin,dest,:DIS)] = rand(150:300)
end

# Finally set all places to have the same capacity
plane_cap = rand(150:200)

# Lets look at a sample demand at random
@show demand[(:BOS,:YYZ,:REG)]

Now let's build the model. We will have our decision variable ``x`` indexed by three things:

1. Origin
2. Destination
3. Class

The upper bound (the demand for each) will vary accordingly.

In [ ]:
nrm2 = Model(solver=GurobiSolver())

@variable(nrm2, 0 <= x[o=airports,
                       d=airports,
                       c=classes; o!=d] <= demand[(o,d,c)])
nrm2

The objective is to maximize the profit we make, summing over each ticket set:

In [ ]:
@objective(nrm2, Max, sum(prices[(o,d,c)]*x[o,d,c] for 
    o in airports, d in airports, c in classes if o != d))
nrm2

Our first set of constraints enforces that all the legs leaving the hub airport must not oversell the plane capacity:

In [ ]:
for d in airports
    if d != :MDW
        println("Adding constraint for hub (MDW) to $d")
        @constraint(nrm2, 
            sum(x[o,d,c] for o in airports, c in classes if o!=d) <= plane_cap)
    end
end
nrm2

Now, as an exercise, complete this model. 

Add constraints that each flight _to_ the hub is not oversold.

Add the objective to maximize revenue 

Finally, solve the model and check the solution